In [1]:
import os
from glob import glob
import pandas as pd
import numpy as np
import copy
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import pandas_profiling 
from itertools import chain


In [7]:
Order = pd.read_csv('data/OrderDataForNTU.csv')
Behave = pd.read_csv('data/BehaviorDataForNTU.csv')


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Data Preprocess  

#### Delete columns that we won't analysis
OperationSystem, SourceType, SearchKeyWord, OnlineMemberId, ProductId  

In [10]:
Behave.drop(columns=['OperationSystem','SourceType','SearchKeyWord',
                          'OnlineMemberId','ProductId'],inplace= True)


#### Datetime format transform

In [11]:
Behave['HitDateTime'] = Behave['HitDateTime'].apply(lambda x: datetime.strptime(x.split('.')[0], '%Y-%m-%d %H:%M:%S'))
Order['TradesDate'] = Order['TradesDate'].apply(lambda x: datetime.strptime(x, '%Y/%m/%d'))


In [15]:
# Generate label encoding column of Hitdate
Behave['HitMonth'] = Behave['HitDateTime'].apply(lambda x:x.month)  
Behave['HitDay'] = Behave['HitDateTime'].apply(lambda x:x.day)
Behave['HitHour'] = Behave['HitDateTime'].apply(lambda x:x.hour)
# Drop hour, minute and second 
Behave['HitDateTime'] = Behave['HitDateTime'].apply(lambda x: datetime(x.year, x.month, x.day))

In [16]:
# Use HitMonth < 10 as train data, others as test data
BehaveTrain = Behave[Behave['HitMonth'] < 10]
BehaveTest = Behave[Behave['HitMonth'] > 10]
BehaveTest = BehaveTest[BehaveTest['VisitorId'].isin(BehaveTrain['VisitorId'].unique())]


### Each user's buying frequency

In [17]:
x = Behave[Behave['TransactionNum'].notna()][['VisitorId','HitDateTime']].sort_values(['VisitorId','HitDateTime'])
y = x.drop_duplicates(subset=['VisitorId','HitDateTime']).groupby('VisitorId').count()
# Users who buy over one time
y[y['HitDateTime']>1]


,HitDateTime
VisitorId,
00029794-73de-40eb-a129-d5e99c6c3c3c,3
000a3865-9931-4a5f-0447-eaf0bd553931,2
0011ff2b-6b7a-47f5-b564-0a3f7f8d8001,2
00190219-041d-4938-8ff7-6254221c279e,2
001d4a98-60b4-49a1-8430-2d8cfb21a732,2
...,...
ffb88e3e-ded3-458e-8137-a188f477fbb6,2
ffe77131-1870-4a07-d860-d7fd3457d873,3
ffe88cec-ccb8-4c9e-a708-5dba732fb6d3,3


In [18]:
# Valid user's transaction time
x[x['VisitorId'].isin(list(y.index))]

,VisitorId,HitDateTime
8533904,00029794-73de-40eb-a129-d5e99c6c3c3c,2018-11-08
6772450,00029794-73de-40eb-a129-d5e99c6c3c3c,2018-11-30
3511346,00029794-73de-40eb-a129-d5e99c6c3c3c,2018-12-11
10218493,0004f0f0-3755-4420-08ca-e6cf95366d7c,2018-11-21
11887958,0005cf08-e922-4e2f-bbcc-22ef96e04d61,2018-04-06
...,...,...
12024741,fffb9811-bacf-43e1-ad0e-04e38ace7d28,2018-07-20
3524196,fffb9811-bacf-43e1-ad0e-04e38ace7d28,2018-10-03
8683054,fffb9811-bacf-43e1-ad0e-04e38ace7d28,2018-10-03
8533857,undefined,2018-03-29
